# SCKAN Example Queries in SPARQL
### This notebook provides a set of example queries to explore SCKAN knowledge base using SPARQL. To use this notebook you will need to install [jupyter notebook](https://jupyter.org/install) with [SPARQL kernel](https://github.com/paulovn/sparql-kernel).
* <b>The comments within the SPARQL queries should provide information on how to customize the queries</b>
* You can copy and paste the queries in Stardog studio or in a query string in your python code to retreive the results in different formats like csv, tsv, or json.
 

## Set up the SPARQL Endpoint for SCKAN 
First, set up the SPARQL endpoint using following magics. To see the list of other available magics run `%lsmagics` in a code section.

In [2]:
%endpoint https://blazegraph.scicrunch.io/blazegraph/sparql 
%format any
%display table
%show 16 
# The "magics" don't have to be in a separate cell from the actual query. 

Endpoint set to: https://blazegraph.scicrunch.io/blazegraph/sparql
Request format: ANY
Display: table
Result maximum size: 16

In [3]:
%lsmagics

Available magics:
%auth  %display  %endpoint  %format  %graph  %header  %http_header  %lang  %load  %log  %lsmagics  %method  %outfile  %prefix  %qparam  %show

%auth (basic|digest|none) <username> <passwd> : send HTTP authentication (use env:<var> to get values from environment variables)
%display raw | table [withtypes] | diagram [svg|png] [withliterals] : set display format
%endpoint <url> : set SPARQL endpoint. **REQUIRED**
%format JSON | N3 | XML | default | any | none : set requested result format
%graph <uri> : set default graph for the queries
%header <string> | OFF : add a persistent SPARQL header line before all queries, or delete all defined headers
%http_header <name> [<value>] : add (or delete) an arbitrary HTTP header to all queries
%lang <lang> [...] | default | all : language(s) preferred for labels
%load <filename> : load a file with magic lines and process them
%log critical | error | warning | info | debug : set logging level
%lsmagics  : list all magics
%method get | post : set HTTP method
%outfile <filename> | off : save raw output to a file (use "%d" in name to add cell number, "off" to cancel saving)
%prefix <name> [<uri>] : set (or delete) a persistent URI prefix for all queries
%qparam <name> [<value>] : add (or delete) a persistent custom parameter to all queries
%show <n> | all : maximum number of shown results

## Example 0: All Neuron Populations
An example query to list all the neuron populations available in NPO along with their labels and preferred labels (if any).

In [4]:
# This query returns the list of all the neuron types available in NPO along with their 
# labels and preferred labels (when available). The query considers each neuron 
# to have at least one connecting region.

PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX ilxtr: <http://uri.interlex.org/tgbugs/uris/readable/>
PREFIX skos: <http://www.w3.org/2004/02/skos/core#>

SELECT DISTINCT ?Neuron_ID ?Neuron_Label ?Neuron_Preferred_Label
{
    ## For partial matching of neuron label or ID. # Replace 'aacar' within quotes.
    # FILTER (CONTAINS (LCase(?Neuron_Label), 'aacar'))
    # FILTER (CONTAINS (LCase(STR(?Neuron_ID)), 'mmset4'))

    # ?Neuron_ID rdfs:subClassOf ilxtr:NeuronSparcNlp.
    
    # We are only interested in connection forming neuron types i.e, neurons that form some connection 
    # between their origin and destination regions
    ?Neuron_ID ilxtr:hasSomaLocation ?A;
               (ilxtr:hasAxonTerminalLocation | ilxtr:hasAxonSensoryLocation) ?B.
    
    ?Neuron_ID rdfs:label ?Neuron_Label;
    OPTIONAL {?Neuron_ID skos:prefLabel ?Neuron_Preferred_Label.}
}
ORDER BY ?Neuron_ID ?Neuron_Label
LIMIT 999

Neuron_ID,Neuron_Label,Neuron_Preferred_Label
http://uri.interlex.org/tgbugs/uris/readable/neuron-type-aacar-1,neuron type aacar 1,
http://uri.interlex.org/tgbugs/uris/readable/neuron-type-aacar-10a,neuron type aacar 10a,
http://uri.interlex.org/tgbugs/uris/readable/neuron-type-aacar-10v,neuron type aacar 10v,
http://uri.interlex.org/tgbugs/uris/readable/neuron-type-aacar-11,neuron type aacar 11,
http://uri.interlex.org/tgbugs/uris/readable/neuron-type-aacar-12,neuron type aacar 12,
http://uri.interlex.org/tgbugs/uris/readable/neuron-type-aacar-13,neuron type aacar 13,
http://uri.interlex.org/tgbugs/uris/readable/neuron-type-aacar-2i,neuron type aacar 2i,
http://uri.interlex.org/tgbugs/uris/readable/neuron-type-aacar-2m,neuron type aacar 2m,
http://uri.interlex.org/tgbugs/uris/readable/neuron-type-aacar-4,neuron type aacar 4,
http://uri.interlex.org/tgbugs/uris/readable/neuron-type-aacar-5,neuron type aacar 5,


## Example 1: Neurons at A projects to B
The most basic query we want to have for SCKAN is to list the neuron populations along with their origins and destonations.  

In [5]:
# Query: return the neuron populations where Neuron at A projects to B. 
# In this case, A refers to the soma location of the neuron population  and B refers to the axon terminal 
# or the axon sensory terminal location of the neuron population.

PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX UBERON: <http://purl.obolibrary.org/obo/UBERON_>
PREFIX partOf: <http://purl.obolibrary.org/obo/BFO_0000050>
PREFIX ilxtr: <http://uri.interlex.org/tgbugs/uris/readable/>

SELECT  ?Neuron_ID ?Neuron_Label ?A ?Region_A ?B ?Region_B
{
  
  ## ***************************** INSTRUCTIONS **************************************** 
  ## UNCOMMENT the lines by removing the hash symbol (#) before FILTER. 
  ## REPLACE the region name on the right side of the equality operator (=) 
  ## by your desired region name within quotes (all in lower case).
  ## ***********************************************************************************

  # FILTER ( LCase(?Region_A) = 'inferior vagus x ganglion' )
  # FILTER ( LCase(?Region_B) = 'epicardium' )

  # FILTER REGEX ( LCase(?Region_A), 'vagus x' ) # For partial matching of Region_A labels  
  # FILTER REGEX ( LCase(?Region_B), 'epicard' ) # For partial matching of Region_B labels  
  
  # FILTER REGEX (?Neuron_Label, 'aacar') # For partial matching of neuron labels

   ?Neuron_ID rdfs:label ?Neuron_Label;
              ilxtr:hasSomaLocation ?A;
              (ilxtr:hasAxonTerminalLocation | ilxtr:hasAxonSensoryLocation) ?B.
 
   ?A rdfs:label ?Region_A. ?B rdfs:label ?Region_B.
}
ORDER BY ?Neuron_ID ?Region_A ?Region_B
LIMIT 1000 

Neuron_ID,Neuron_Label,A,Region_A,B,Region_B
http://uri.interlex.org/tgbugs/uris/readable/neuron-type-aacar-1,neuron type aacar 1,http://purl.obolibrary.org/obo/UBERON_0009050,nucleus of solitary tract,http://purl.obolibrary.org/obo/UBERON_0002022,insula
http://uri.interlex.org/tgbugs/uris/readable/neuron-type-aacar-10a,neuron type aacar 10a,http://uri.interlex.org/base/ilx_0793555,Atrial intrinsic cardiac ganglion,http://uri.interlex.org/base/ilx_0793555,Atrial intrinsic cardiac ganglion
http://uri.interlex.org/tgbugs/uris/readable/neuron-type-aacar-10v,neuron type aacar 10v,http://uri.interlex.org/base/ilx_0793556,ventricular intrinsic cardiac ganglion,http://uri.interlex.org/base/ilx_0793556,ventricular intrinsic cardiac ganglion
http://uri.interlex.org/tgbugs/uris/readable/neuron-type-aacar-11,neuron type aacar 11,http://purl.obolibrary.org/obo/UBERON_0005363,inferior vagus X ganglion,http://purl.obolibrary.org/obo/UBERON_0000947,aorta
http://uri.interlex.org/tgbugs/uris/readable/neuron-type-aacar-11,neuron type aacar 11,http://purl.obolibrary.org/obo/UBERON_0005363,inferior vagus X ganglion,http://purl.obolibrary.org/obo/UBERON_0002348,epicardium
http://uri.interlex.org/tgbugs/uris/readable/neuron-type-aacar-11,neuron type aacar 11,http://purl.obolibrary.org/obo/UBERON_0005363,inferior vagus X ganglion,http://purl.obolibrary.org/obo/UBERON_0002084,heart left ventricle
http://uri.interlex.org/tgbugs/uris/readable/neuron-type-aacar-11,neuron type aacar 11,http://purl.obolibrary.org/obo/UBERON_0005363,inferior vagus X ganglion,http://purl.obolibrary.org/obo/UBERON_0002079,left cardiac atrium
http://uri.interlex.org/tgbugs/uris/readable/neuron-type-aacar-11,neuron type aacar 11,http://purl.obolibrary.org/obo/UBERON_0005363,inferior vagus X ganglion,http://purl.obolibrary.org/obo/UBERON_0009050,nucleus of solitary tract
http://uri.interlex.org/tgbugs/uris/readable/neuron-type-aacar-11,neuron type aacar 11,http://purl.obolibrary.org/obo/UBERON_0005363,inferior vagus X ganglion,http://purl.obolibrary.org/obo/UBERON_0002012,pulmonary artery
http://uri.interlex.org/tgbugs/uris/readable/neuron-type-aacar-11,neuron type aacar 11,http://purl.obolibrary.org/obo/UBERON_0005363,inferior vagus X ganglion,http://purl.obolibrary.org/obo/UBERON_0002078,right cardiac atrium


## Example 2: Neuron Population at A projects to B (with synonyms)
This is the same query as before except that we can filter through any synonyms of the regions instead of just exact labels.

In [6]:
# Query: return the neuron populations where Neuron at A projects to B with synonyms for A and B. 
# In this case, A refers to the soma location of the neuron population  and B refers to the axon terminal 
# or the axon sensory terminal location of the neuron population. 

PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX partOf: <http://purl.obolibrary.org/obo/BFO_0000050>
PREFIX ilxtr: <http://uri.interlex.org/tgbugs/uris/readable/>
PREFIX oboInOwl: <http://www.geneontology.org/formats/oboInOwl#>

SELECT  ?Neuron_ID ?Neuron_Label ?A ?Region_A ?B ?Region_B
{
  
  ## ***************************** INSTRUCTIONS **************************************** 
  ## UNCOMMENT the lines by removing the hash symbol (#) before FILTER. 
  ## REPLACE the region name on the right side of the equality operator (=) 
  ## by your desired region name within quotes (all in lower case).
  ## ***********************************************************************************

  # FILTER ( LCase(?Region_A) = 'nodose ganglion' )
  # FILTER ( LCase(?Region_B) = 'epicardium' )

  # FILTER REGEX ( LCase(?Region_A), 'nodose' )  # For partial matching of Region_A labels  
  # FILTER REGEX ( LCase(?Region_B), 'epicard' ) # For partial matching of Region_B labels  
  
  # FILTER REGEX (?Neuron_Label, 'aacar') # For partial matching of neuron labels

   ?Neuron_ID rdfs:label ?Neuron_Label;
              ilxtr:hasSomaLocation ?A;
              (ilxtr:hasAxonTerminalLocation | ilxtr:hasAxonSensoryLocation) ?B.
 
   ?A (rdfs:label | oboInOwl:hasExactSynonym) ?Region_A. 
   ?B (rdfs:label | oboInOwl:hasExactSynonym) ?Region_B.       
}
ORDER BY ?Neuron_ID ?Region_A ?Region_B
LIMIT 9999

Neuron_ID,Neuron_Label,A,Region_A,B,Region_B
http://uri.interlex.org/tgbugs/uris/readable/neuron-type-aacar-1,neuron type aacar 1,http://purl.obolibrary.org/obo/UBERON_0009050,nucleus of solitary tract,http://purl.obolibrary.org/obo/UBERON_0002022,central lobe
http://uri.interlex.org/tgbugs/uris/readable/neuron-type-aacar-1,neuron type aacar 1,http://purl.obolibrary.org/obo/UBERON_0009050,nucleus of solitary tract,http://purl.obolibrary.org/obo/UBERON_0002022,cortex of island
http://uri.interlex.org/tgbugs/uris/readable/neuron-type-aacar-1,neuron type aacar 1,http://purl.obolibrary.org/obo/UBERON_0009050,nucleus of solitary tract,http://purl.obolibrary.org/obo/UBERON_0002022,insula
http://uri.interlex.org/tgbugs/uris/readable/neuron-type-aacar-1,neuron type aacar 1,http://purl.obolibrary.org/obo/UBERON_0009050,nucleus of solitary tract,http://purl.obolibrary.org/obo/UBERON_0002022,insula lobule
http://uri.interlex.org/tgbugs/uris/readable/neuron-type-aacar-1,neuron type aacar 1,http://purl.obolibrary.org/obo/UBERON_0009050,nucleus of solitary tract,http://purl.obolibrary.org/obo/UBERON_0002022,insular lobe
http://uri.interlex.org/tgbugs/uris/readable/neuron-type-aacar-1,neuron type aacar 1,http://purl.obolibrary.org/obo/UBERON_0009050,nucleus of solitary tract,http://purl.obolibrary.org/obo/UBERON_0002022,insular region
http://uri.interlex.org/tgbugs/uris/readable/neuron-type-aacar-1,neuron type aacar 1,http://purl.obolibrary.org/obo/UBERON_0009050,nucleus of solitary tract,http://purl.obolibrary.org/obo/UBERON_0002022,island of Reil
http://uri.interlex.org/tgbugs/uris/readable/neuron-type-aacar-1,neuron type aacar 1,http://purl.obolibrary.org/obo/UBERON_0009050,nucleus of solitary tract,http://purl.obolibrary.org/obo/UBERON_0002022,lobus insularis
http://uri.interlex.org/tgbugs/uris/readable/neuron-type-aacar-1,neuron type aacar 1,http://purl.obolibrary.org/obo/UBERON_0009050,nucleus of the tractus solitarius,http://purl.obolibrary.org/obo/UBERON_0002022,central lobe
http://uri.interlex.org/tgbugs/uris/readable/neuron-type-aacar-1,neuron type aacar 1,http://purl.obolibrary.org/obo/UBERON_0009050,nucleus of the tractus solitarius,http://purl.obolibrary.org/obo/UBERON_0002022,cortex of island


## Example 3: Neurons at A projects to B via C
One of the most important queries we frequently need is to list the neuron populations along with their origin (A), destination (B), and via locations (C). 

In [7]:
# QUERY: List the Neuron Populations where Neurons at Region A Projects to B via C.
# This query returns any neuron population that has its soma located in Region A,
#                                                       axon terminal or axon sensory terminal located in Region B, and 
#                                                       axon located in region C.

PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX partOf: <http://purl.obolibrary.org/obo/BFO_0000050>
PREFIX ilxtr: <http://uri.interlex.org/tgbugs/uris/readable/>

SELECT DISTINCT ?Neuron_ID ?Neuron_Label ?A ?Region_A ?B ?Region_B ?C ?Region_C
{
   ## ***************************** INSTRUCTIONS **************************************** 
   ## UNCOMMENT the lines by removing the hash symbol (#) before FILTER. 
   ## REPLACE the region name on the right side of the equality operator (=) 
   ## by your desired region name within quotes (all in lower case).
   ## ***********************************************************************************  

   ## For exact matching of the regions names
   # FILTER ( LCase(?Region_A) = 'nucleus ambiguus' )
   # FILTER ( LCase(?Region_B) = 'bronchiole parasympathetic ganglia' )
   # FILTER ( LCase(?Region_C) = 'vagus nerve' )
   
   ## For partial matching of region names (label or synonyms)
   # FILTER REGEX ( LCase(?Region_A), 'ambiguus' )
   # FILTER REGEX ( LCase(?Region_B), 'bronchiole parasympathetic' ) 
   # FILTER REGEX ( LCase(?Region_C), 'vagus' )
   
   # FILTER REGEX ( LCase(?Neuron_Label), 'bolew unbranched 25' ) # For partial matching of neuron label
                                                      # Replace 'bolew unbranched 25' within quotes.  

   ?Neuron_ID rdfs:label ?Neuron_Label;
               ilxtr:hasSomaLocation ?A;
               ilxtr:hasAxonLocation ?C;
              (ilxtr:hasAxonTerminalLocation | ilxtr:hasAxonSensoryLocation) ?B. 
    
          ?A rdfs:label ?Region_A.
          ?B rdfs:label ?Region_B.
          ?C rdfs:label ?Region_C. 
}
ORDER BY ?Neuron_ID ?Region_A ?Region_B ?Region_C
LIMIT 5000

Neuron_ID,Neuron_Label,A,Region_A,B,Region_B,C,Region_C
http://uri.interlex.org/tgbugs/uris/readable/neuron-type-aacar-1,neuron type aacar 1,http://purl.obolibrary.org/obo/UBERON_0009050,nucleus of solitary tract,http://purl.obolibrary.org/obo/UBERON_0002022,insula,http://purl.obolibrary.org/obo/UBERON_0000988,Pons
http://uri.interlex.org/tgbugs/uris/readable/neuron-type-aacar-1,neuron type aacar 1,http://purl.obolibrary.org/obo/UBERON_0009050,nucleus of solitary tract,http://purl.obolibrary.org/obo/UBERON_0002022,insula,http://purl.obolibrary.org/obo/UBERON_0001894,diencephalon
http://uri.interlex.org/tgbugs/uris/readable/neuron-type-aacar-1,neuron type aacar 1,http://purl.obolibrary.org/obo/UBERON_0009050,nucleus of solitary tract,http://purl.obolibrary.org/obo/UBERON_0002022,insula,http://purl.obolibrary.org/obo/UBERON_0001891,midbrain
http://uri.interlex.org/tgbugs/uris/readable/neuron-type-aacar-1,neuron type aacar 1,http://purl.obolibrary.org/obo/UBERON_0009050,nucleus of solitary tract,http://purl.obolibrary.org/obo/UBERON_0002022,insula,http://purl.obolibrary.org/obo/UBERON_0000988,pons
http://uri.interlex.org/tgbugs/uris/readable/neuron-type-aacar-11,neuron type aacar 11,http://purl.obolibrary.org/obo/UBERON_0005363,inferior vagus X ganglion,http://purl.obolibrary.org/obo/UBERON_0000947,aorta,http://purl.obolibrary.org/obo/UBERON_0001896,medulla oblongata
http://uri.interlex.org/tgbugs/uris/readable/neuron-type-aacar-11,neuron type aacar 11,http://purl.obolibrary.org/obo/UBERON_0005363,inferior vagus X ganglion,http://purl.obolibrary.org/obo/UBERON_0002348,epicardium,http://purl.obolibrary.org/obo/UBERON_0001896,medulla oblongata
http://uri.interlex.org/tgbugs/uris/readable/neuron-type-aacar-11,neuron type aacar 11,http://purl.obolibrary.org/obo/UBERON_0005363,inferior vagus X ganglion,http://purl.obolibrary.org/obo/UBERON_0002084,heart left ventricle,http://purl.obolibrary.org/obo/UBERON_0001896,medulla oblongata
http://uri.interlex.org/tgbugs/uris/readable/neuron-type-aacar-11,neuron type aacar 11,http://purl.obolibrary.org/obo/UBERON_0005363,inferior vagus X ganglion,http://purl.obolibrary.org/obo/UBERON_0002079,left cardiac atrium,http://purl.obolibrary.org/obo/UBERON_0001896,medulla oblongata
http://uri.interlex.org/tgbugs/uris/readable/neuron-type-aacar-11,neuron type aacar 11,http://purl.obolibrary.org/obo/UBERON_0005363,inferior vagus X ganglion,http://purl.obolibrary.org/obo/UBERON_0009050,nucleus of solitary tract,http://purl.obolibrary.org/obo/UBERON_0001896,medulla oblongata
http://uri.interlex.org/tgbugs/uris/readable/neuron-type-aacar-11,neuron type aacar 11,http://purl.obolibrary.org/obo/UBERON_0005363,inferior vagus X ganglion,http://purl.obolibrary.org/obo/UBERON_0002012,pulmonary artery,http://purl.obolibrary.org/obo/UBERON_0001896,medulla oblongata


### Example 4: Neurons at A projects to B via C (with synonyms)
This is the same query as the previous except that we can now filter through any synonyms of the regions instead of just exact labels. 

In [13]:
# QUERY: List the Neuron Populations where Neuron at Region A Projects to B via C.
# This query returns any neuron population that has its soma located in Region A,
#                                                       axon terminal or axon sensory terminal located in Region B, and 
#                                                       axon located in region C.

PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX partOf: <http://purl.obolibrary.org/obo/BFO_0000050>
PREFIX ilxtr: <http://uri.interlex.org/tgbugs/uris/readable/>
PREFIX oboInOwl: <http://www.geneontology.org/formats/oboInOwl#> 

SELECT DISTINCT ?Neuron_ID ?Neuron_Label ?A ?Region_A ?B ?Region_B ?C ?Region_C
{
   ## ***************************** INSTRUCTIONS **************************************** 
   ## UNCOMMENT the lines by removing the hash symbol (#) before FILTER. 
   ## REPLACE the region name on the right side of the equality operator (=) 
   ## by your desired region name within quotes (all in lower case).
   ## ***********************************************************************************  

   ## For exact matching of the regions names
   # FILTER ( LCase(?Region_A) = 'nucleus ambiguus' )
   # FILTER ( LCase(?Region_B) = 'bronchiole parasympathetic ganglia' )
   # FILTER ( LCase(?Region_C) = 'vagus nerve' )
   
   ## For partial matching of region names (label or synonyms)
   # FILTER REGEX ( LCase(?Region_A), 'ambiguus' )
   # FILTER REGEX ( LCase(?Region_B), 'bronchiole parasympathetic' ) 
   # FILTER REGEX ( LCase(?Region_C), 'vagus' )
   
   ## For partial matching of neuron label. Replace 'splen 4' within quotes.
   # FILTER REGEX (?Neuron_Label, 'splen 4') 
   
   ?Neuron_ID rdfs:label ?Neuron_Label;
               ilxtr:hasSomaLocation ?A;
               ilxtr:hasAxonLocation ?C;
              (ilxtr:hasAxonTerminalLocation | ilxtr:hasAxonSensoryLocation) ?B. 
    
    ?A (rdfs:label | oboInOwl:hasExactSynonym) ?Region_A.
    ?B (rdfs:label | oboInOwl:hasExactSynonym) ?Region_B.
    ?C (rdfs:label | oboInOwl:hasExactSynonym) ?Region_C.
}
ORDER BY ?Neuron_ID ?Region_A ?Region_B ?Region_C
LIMIT 500

Neuron_ID,Neuron_Label,A,Region_A,B,Region_B,C,Region_C
http://uri.interlex.org/tgbugs/uris/readable/neuron-type-aacar-1,neuron type aacar 1,http://purl.obolibrary.org/obo/UBERON_0009050,nucleus of solitary tract,http://purl.obolibrary.org/obo/UBERON_0002022,central lobe,http://purl.obolibrary.org/obo/UBERON_0001894,between brain
http://uri.interlex.org/tgbugs/uris/readable/neuron-type-aacar-1,neuron type aacar 1,http://purl.obolibrary.org/obo/UBERON_0009050,nucleus of solitary tract,http://purl.obolibrary.org/obo/UBERON_0002022,central lobe,http://purl.obolibrary.org/obo/UBERON_0001894,diencephalon
http://uri.interlex.org/tgbugs/uris/readable/neuron-type-aacar-1,neuron type aacar 1,http://purl.obolibrary.org/obo/UBERON_0009050,nucleus of solitary tract,http://purl.obolibrary.org/obo/UBERON_0002022,central lobe,http://purl.obolibrary.org/obo/UBERON_0001894,interbrain
http://uri.interlex.org/tgbugs/uris/readable/neuron-type-aacar-1,neuron type aacar 1,http://purl.obolibrary.org/obo/UBERON_0009050,nucleus of solitary tract,http://purl.obolibrary.org/obo/UBERON_0002022,central lobe,http://purl.obolibrary.org/obo/UBERON_0001894,mature diencephalon
http://uri.interlex.org/tgbugs/uris/readable/neuron-type-aacar-1,neuron type aacar 1,http://purl.obolibrary.org/obo/UBERON_0009050,nucleus of solitary tract,http://purl.obolibrary.org/obo/UBERON_0002022,central lobe,http://purl.obolibrary.org/obo/UBERON_0001891,midbrain
http://uri.interlex.org/tgbugs/uris/readable/neuron-type-aacar-1,neuron type aacar 1,http://purl.obolibrary.org/obo/UBERON_0009050,nucleus of solitary tract,http://purl.obolibrary.org/obo/UBERON_0002022,central lobe,http://purl.obolibrary.org/obo/UBERON_0000988,pons
http://uri.interlex.org/tgbugs/uris/readable/neuron-type-aacar-1,neuron type aacar 1,http://purl.obolibrary.org/obo/UBERON_0009050,nucleus of solitary tract,http://purl.obolibrary.org/obo/UBERON_0002022,central lobe,http://purl.obolibrary.org/obo/UBERON_0000988,pons Varolii
http://uri.interlex.org/tgbugs/uris/readable/neuron-type-aacar-1,neuron type aacar 1,http://purl.obolibrary.org/obo/UBERON_0009050,nucleus of solitary tract,http://purl.obolibrary.org/obo/UBERON_0002022,central lobe,http://purl.obolibrary.org/obo/UBERON_0000988,pons of Varolius
http://uri.interlex.org/tgbugs/uris/readable/neuron-type-aacar-1,neuron type aacar 1,http://purl.obolibrary.org/obo/UBERON_0009050,nucleus of solitary tract,http://purl.obolibrary.org/obo/UBERON_0002022,central lobe,http://purl.obolibrary.org/obo/UBERON_0001894,thalamencephalon
http://uri.interlex.org/tgbugs/uris/readable/neuron-type-aacar-1,neuron type aacar 1,http://purl.obolibrary.org/obo/UBERON_0009050,nucleus of solitary tract,http://purl.obolibrary.org/obo/UBERON_0002022,cortex of island,http://purl.obolibrary.org/obo/UBERON_0001894,between brain


## Example 5: Neurons at A projects to B via Nerve C
This is another important query for SCKAN where we want to know the neuronal connections based on their via locations in nerves rather than just any axon locations.

In [8]:
# QUERY: List the Neuron Populations where Neuron at Region A Projects to B via Nerve C.
# This query returns any neuron population that has its soma located in Region A,
#                                                       axon terminal located in Region B, and 
#                                                       axon located in nerve C.
# Note that region C could be of type nerve, nerve plexus, or nerve fiber from UBERON.

PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX partOf: <http://purl.obolibrary.org/obo/BFO_0000050>
PREFIX ilxtr: <http://uri.interlex.org/tgbugs/uris/readable/>

# Neuron populations where A projects to B via some Nerve C

SELECT DISTINCT ?Neuron_ID ?Neuron_Label ?A ?Region_A ?B ?Region_B ?C ?Region_C 
{
   ## ***************************** INSTRUCTIONS **************************************** 
   ## UNCOMMENT the lines by removing the hash symbol (#) before FILTER. 
   ## REPLACE the region name on the right side of the equality operator (=) 
   ## by your desired region name within quotes (all in lower case).
   ## ***********************************************************************************
 
   ## For exact matching of region names 
   # FILTER ( LCase(?Region_A) = 'inferior vagus x ganglion' )
   # FILTER ( LCase(?Region_B) = 'nucleus of solitary tract' )
   # FILTER ( LCase(?Region_C) = 'vagus nerve' )
   
   ## For partial matching of region names 
   # FILTER REGEX (LCase(?Region_A), 'vagus x' )
   # FILTER REGEX (LCase(?Region_B), 'aortic' )
   # FILTER REGEX (LCase(?Region_C), 'vagus' )
   
   ## For partial matching of neuron label pr ID. # Replace 'splen 4' within quotes.
   # FILTER REGEX (?Neuron_Label, 'splen 4')
   # FILTER REGEX (LCase(?Neuron_Label), 'main clilary ganglion')
   # FILTER REGEX (LCASE(STR(?Neuron_ID)), 'mmset4')   

   
    ?Neuron_ID rdfs:label ?Neuron_Label;
       ilxtr:hasSomaLocation ?A;
       ilxtr:hasAxonLocation ?C;
      (ilxtr:hasAxonTerminalLocation | ilxtr:hasAxonSensoryLocation) ?B.
    
    ?A rdfs:label ?Region_A.
    ?B rdfs:label ?Region_B.
    ?C rdfs:label ?Region_C.
    
    ## Check if region C is a subclass+ of a 'nerve' or 'nerve plexus'.
    ?C (rdfs:subClassOf+/rdfs:label) | rdfs:label ?nerve_label.  
    ## We have FMA 'Nerve' and UBERON 'nerve'. Also including UBERON 'nerve plexus'.
    FILTER (?nerve_label in ('nerve', 'nerve plexus', 'nerve fiber'))
                   # || CONTAINS(LCASE(?nerve_label), 'nerve'))
	
}
ORDER BY ?Neuron_ID ?Region_B ?Region_C
LIMIT 5000

Neuron_ID,Neuron_Label,A,Region_A,B,Region_B,C,Region_C
http://uri.interlex.org/tgbugs/uris/readable/neuron-type-aacar-5,neuron type aacar 5,http://purl.obolibrary.org/obo/UBERON_0006459,T3 segment of thoracic spinal cord,http://uri.interlex.org/base/ilx_0793555,Atrial intrinsic cardiac ganglion,http://uri.interlex.org/base/ilx_0793211,white communicating ramus of fifth thoracic spinal nerve
http://uri.interlex.org/tgbugs/uris/readable/neuron-type-aacar-5,neuron type aacar 5,http://purl.obolibrary.org/obo/UBERON_0006452,T4 segment of thoracic spinal cord,http://uri.interlex.org/base/ilx_0793555,Atrial intrinsic cardiac ganglion,http://uri.interlex.org/base/ilx_0793211,white communicating ramus of fifth thoracic spinal nerve
http://uri.interlex.org/tgbugs/uris/readable/neuron-type-aacar-5,neuron type aacar 5,http://purl.obolibrary.org/obo/UBERON_0006454,T6 segment of thoracic spinal cord,http://uri.interlex.org/base/ilx_0793555,Atrial intrinsic cardiac ganglion,http://uri.interlex.org/base/ilx_0793211,white communicating ramus of fifth thoracic spinal nerve
http://uri.interlex.org/tgbugs/uris/readable/neuron-type-aacar-5,neuron type aacar 5,http://purl.obolibrary.org/obo/UBERON_0006453,T5 segment of thoracic spinal cord,http://uri.interlex.org/base/ilx_0793555,Atrial intrinsic cardiac ganglion,http://uri.interlex.org/base/ilx_0793211,white communicating ramus of fifth thoracic spinal nerve
http://uri.interlex.org/tgbugs/uris/readable/neuron-type-aacar-5,neuron type aacar 5,http://purl.obolibrary.org/obo/UBERON_0006458,T2 segment of thoracic spinal cord,http://uri.interlex.org/base/ilx_0793555,Atrial intrinsic cardiac ganglion,http://uri.interlex.org/base/ilx_0793211,white communicating ramus of fifth thoracic spinal nerve
http://uri.interlex.org/tgbugs/uris/readable/neuron-type-aacar-5,neuron type aacar 5,http://purl.obolibrary.org/obo/UBERON_0006457,T1 segment of thoracic spinal cord,http://uri.interlex.org/base/ilx_0793555,Atrial intrinsic cardiac ganglion,http://uri.interlex.org/base/ilx_0793211,white communicating ramus of fifth thoracic spinal nerve
http://uri.interlex.org/tgbugs/uris/readable/neuron-type-aacar-5,neuron type aacar 5,http://purl.obolibrary.org/obo/UBERON_0006458,T2 segment of thoracic spinal cord,http://uri.interlex.org/base/ilx_0793555,Atrial intrinsic cardiac ganglion,http://uri.interlex.org/base/ilx_0738372,white communicating ramus of first thoracic spinal nerve
http://uri.interlex.org/tgbugs/uris/readable/neuron-type-aacar-5,neuron type aacar 5,http://purl.obolibrary.org/obo/UBERON_0006452,T4 segment of thoracic spinal cord,http://uri.interlex.org/base/ilx_0793555,Atrial intrinsic cardiac ganglion,http://uri.interlex.org/base/ilx_0738372,white communicating ramus of first thoracic spinal nerve
http://uri.interlex.org/tgbugs/uris/readable/neuron-type-aacar-5,neuron type aacar 5,http://purl.obolibrary.org/obo/UBERON_0006457,T1 segment of thoracic spinal cord,http://uri.interlex.org/base/ilx_0793555,Atrial intrinsic cardiac ganglion,http://uri.interlex.org/base/ilx_0738372,white communicating ramus of first thoracic spinal nerve
http://uri.interlex.org/tgbugs/uris/readable/neuron-type-aacar-5,neuron type aacar 5,http://purl.obolibrary.org/obo/UBERON_0006454,T6 segment of thoracic spinal cord,http://uri.interlex.org/base/ilx_0793555,Atrial intrinsic cardiac ganglion,http://uri.interlex.org/base/ilx_0738372,white communicating ramus of first thoracic spinal nerve


## Example 6: Neurons with All Connected Regions
This query returns a list of neuron populations along with their connected regions based on the their locational phenotypes. 

In [9]:
# Query: This query returns neuron populations with their connecting regions.
# The regions could be any of soma, axon, axon terminal, dendrite, or axon sensory terminal locations.

PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX partOf: <http://purl.obolibrary.org/obo/BFO_0000050>
PREFIX ilxtr: <http://uri.interlex.org/tgbugs/uris/readable/>
PREFIX oboInOwl: <http://www.geneontology.org/formats/oboInOwl#>

SELECT DISTINCT ?Neuron_ID ?Neuron_Label ?Connection_Type ?Location_ID ?Location_Label
{
  ## ***************************** INSTRUCTIONS **************************************** 
  ## UNCOMMENT the lines by removing the hash symbol (#) before FILTER. 
  ## REPLACE the region name on the right side of the equality operator (=) 
  ## by your desired region name within quotes (all in lower case).
  ## ***********************************************************************************
   
  ## For exact matching of the region name
  # FILTER (LCase(?Location_Label) = 'inferior vagus x ganglion')

  ## For partial matching of location name
  # FILTER REGEX (LCase(?Location_Label), 'vagus x')

  ## For partial matching of neuron label. Replace 'bromo' by other types.  
  #  FILTER REGEX (LCase(?Neuron_Label), 'bromo') 
    
    # We are only interested in connection forming neuron types i.e, neurons that form some connection 
    # between their origin and destination regions
    ?Neuron_ID ilxtr:hasSomaLocation ?A;
               (ilxtr:hasAxonTerminalLocation | ilxtr:hasAxonSensoryLocation) ?B.

   
    ?Neuron_ID ilxtr:hasConnectedLocation ?Location_ID;
               rdfs:label ?Neuron_Label.
    
    ?Location_ID rdfs:label ?Location_Label.
    ?Neuron_ID ?Connection_Type ?Location_ID.
    
    FILTER (?Connection_Type != ilxtr:hasConnectedLocation)
}
ORDER BY ?Neuron_ID DESC(?Connection_Type) ?Location_Label
LIMIT 5000

Neuron_ID,Neuron_Label,Connection_Type,Location_ID,Location_Label
http://uri.interlex.org/tgbugs/uris/readable/neuron-type-aacar-1,neuron type aacar 1,http://uri.interlex.org/tgbugs/uris/readable/hasSomaLocation,http://purl.obolibrary.org/obo/UBERON_0009050,nucleus of solitary tract
http://uri.interlex.org/tgbugs/uris/readable/neuron-type-aacar-1,neuron type aacar 1,http://uri.interlex.org/tgbugs/uris/readable/hasAxonTerminalLocation,http://purl.obolibrary.org/obo/UBERON_0002022,insula
http://uri.interlex.org/tgbugs/uris/readable/neuron-type-aacar-1,neuron type aacar 1,http://uri.interlex.org/tgbugs/uris/readable/hasAxonLocation,http://purl.obolibrary.org/obo/UBERON_0000988,Pons
http://uri.interlex.org/tgbugs/uris/readable/neuron-type-aacar-1,neuron type aacar 1,http://uri.interlex.org/tgbugs/uris/readable/hasAxonLocation,http://purl.obolibrary.org/obo/UBERON_0001894,diencephalon
http://uri.interlex.org/tgbugs/uris/readable/neuron-type-aacar-1,neuron type aacar 1,http://uri.interlex.org/tgbugs/uris/readable/hasAxonLocation,http://purl.obolibrary.org/obo/UBERON_0001891,midbrain
http://uri.interlex.org/tgbugs/uris/readable/neuron-type-aacar-1,neuron type aacar 1,http://uri.interlex.org/tgbugs/uris/readable/hasAxonLocation,http://purl.obolibrary.org/obo/UBERON_0000988,pons
http://uri.interlex.org/tgbugs/uris/readable/neuron-type-aacar-10a,neuron type aacar 10a,http://uri.interlex.org/tgbugs/uris/readable/hasSomaLocation,http://uri.interlex.org/base/ilx_0793555,Atrial intrinsic cardiac ganglion
http://uri.interlex.org/tgbugs/uris/readable/neuron-type-aacar-10a,neuron type aacar 10a,http://uri.interlex.org/tgbugs/uris/readable/hasAxonTerminalLocation,http://uri.interlex.org/base/ilx_0793555,Atrial intrinsic cardiac ganglion
http://uri.interlex.org/tgbugs/uris/readable/neuron-type-aacar-10v,neuron type aacar 10v,http://uri.interlex.org/tgbugs/uris/readable/hasSomaLocation,http://uri.interlex.org/base/ilx_0793556,ventricular intrinsic cardiac ganglion
http://uri.interlex.org/tgbugs/uris/readable/neuron-type-aacar-10v,neuron type aacar 10v,http://uri.interlex.org/tgbugs/uris/readable/hasAxonTerminalLocation,http://uri.interlex.org/base/ilx_0793556,ventricular intrinsic cardiac ganglion


## Example 7: Neurons with Metadata
This is a query to list neuron populations with their available metadata in SCKAN such as species, sex, phenotype, and reference whenever available.  

In [10]:
# Query to retreive metadata of the neuron populations.

PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX partOf: <http://purl.obolibrary.org/obo/BFO_0000050>
PREFIX ilxtr: <http://uri.interlex.org/tgbugs/uris/readable/>
PREFIX skos: <http://www.w3.org/2004/02/skos/core#>

SELECT DISTINCT ?Neuron_IRI ?Neuron_Label ?Neuron_Pref_Label ?Species ?Sex 
                ?Phenotypes ?Forward_Connections ?Alert ?Reference
WHERE
{
    { 
        SELECT DISTINCT  ?Neuron_IRI ?Neuron_Label ?Neuron_Pref_Label ?Sex ?Alert ?Reference
        WHERE                  
        {
            ?Neuron_IRI rdfs:subClassOf* SAO:1417703748. #http://uri.neuinfo.org/nif/nifstd/sao1417703748   
            OPTIONAL{?Neuron_IRI rdfs:label ?Neuron_Label.}
            OPTIONAL{?Neuron_IRI skos:prefLabel ?Neuron_Pref_Label.}

            ?Neuron_IRI ilxtr:hasSomaLocation ?A_IRI.
            OPTIONAL {?Neuron_IRI ilxtr:hasAxonLocation ?C_IRI.}
            ?Neuron_IRI (ilxtr:hasAxonTerminalLocation | ilxtr:hasAxonSensoryLocation) ?B_IRI.
                    
            OPTIONAL {?Neuron_IRI ilxtr:hasPhenotypicSex/rdfs:label ?Sex.}
            OPTIONAL {?Neuron_IRI ilxtr:reference ?Reference.}
            OPTIONAL {?Neuron_IRI ilxtr:alertNote ?Alert.}
        }
    }
    
    { 
        SELECT DISTINCT  ?Neuron_IRI
        (group_concat(distinct ?ObservedIn; separator=", ") as ?Species) 
        WHERE                  
        {
            ?Neuron_IRI rdfs:subClassOf* SAO:1417703748. # Neuron URI http://uri.neuinfo.org/nif/nifstd/sao1417703748   
            
            ?Neuron_IRI ilxtr:hasSomaLocation ?A_IRI.
            OPTIONAL {?Neuron_IRI ilxtr:hasAxonLocation ?C_IRI.}
            ?Neuron_IRI (ilxtr:hasAxonTerminalLocation | ilxtr:hasAxonSensoryLocation) ?B_IRI.
                    
            OPTIONAL {?Neuron_IRI ilxtr:isObservedInSpecies/rdfs:label ?ObservedIn.}
        }
        GROUP BY ?Neuron_IRI
    }
    {
        SELECT DISTINCT  ?Neuron_IRI 
        (group_concat(distinct ?ForwardConnection; separator=", ") as ?Forward_Connections)  
        WHERE                  
        {
            ?Neuron_IRI rdfs:subClassOf+ SAO:1417703748. #http://uri.neuinfo.org/nif/nifstd/sao1417703748   
            
            ?Neuron_IRI ilxtr:hasSomaLocation ?A_IRI.
            OPTIONAL {?Neuron_IRI ilxtr:hasAxonLocation ?C_IRI.}
            ?Neuron_IRI (ilxtr:hasAxonTerminalLocation | ilxtr:hasAxonSensoryLocation) ?B_IRI.

            OPTIONAL {?Neuron_IRI ilxtr:hasForwardConnection ?ForwardConnection.}
        }
        GROUP BY ?Neuron_IRI
    }

    {
        SELECT DISTINCT  ?Neuron_IRI 
        (group_concat(distinct ?Phenotype; separator=", ") as ?Phenotypes) 
        WHERE                  
        {
            ?Neuron_IRI rdfs:subClassOf* SAO:1417703748. #http://uri.neuinfo.org/nif/nifstd/sao1417703748   
            
            ?Neuron_IRI ilxtr:hasSomaLocation ?A_IRI.
            OPTIONAL {?Neuron_IRI ilxtr:hasAxonLocation ?C_IRI.}
            ?Neuron_IRI (ilxtr:hasAxonTerminalLocation | ilxtr:hasAxonSensoryLocation) ?B_IRI.

            OPTIONAL {?Neuron_IRI (ilxtr:hasNeuronalPhenotype | 
                                  ilxtr:hasFunctionalCircuitRole |
                                  ilxtr:hasCircuitRole |
                                  ilxtr:hasProjection  
                                  )/rdfs:label ?Phenotype.}
        }
        GROUP BY ?Neuron_IRI
    }
}
ORDER BY ?Neuron_IRI ?Neuron_Label
LIMIT 500

Error: Response processing error: 'NoneType' object has no attribute 'replace'

## Example 8: Neurons Targetting Major Organs
We often want to know list the neuron populations based on their targets in major organs. This query looks for the terminals of the populations that reside in any part of a major organ. In this example, we ask about the populations that terminate in any part of heart.   

In [11]:
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX partOf: <http://purl.obolibrary.org/obo/BFO_0000050>
PREFIX ilxtr: <http://uri.interlex.org/tgbugs/uris/readable/>

SELECT DISTINCT ?Neuron_ID ?A_IRI ?A_Label ?B_IRI ?B_Label ?Target_Organ
{                
    
    ## For exact matching of the target organ name
    FILTER (LCase(?Target_Organ) = 'heart')

    ## For partial matching of the target organ name
    # FILTER REGEX (LCase(?Target_Organ), 'heart left')

    
   # ?Neuron_ID rdfs:subClassOf+/rdfs:label 'Neuron'. #http://uri.neuinfo.org/nif/nifstd/sao1417703748
   ?Neuron_ID rdfs:subClassOf+ SAO:1417703748. 
    
    ?Neuron_ID ilxtr:hasSomaLocation ?A_IRI.
            ?A_IRI (rdfs:label) ?A_Label.
   
   
    ?Neuron_ID (ilxtr:hasAxonTerminalLocation | ilxtr:hasAxonSensoryLocation) ?B_IRI.
                ?B_IRI (rdfs:label) ?B_Label.

    ?B_IRI rdfs:subClassOf+ [rdf:type owl:Restriction ;
                                    owl:onProperty partOf: ; 
                                    owl:someValuesFrom/rdfs:label ?Target_Organ].
    
                    
    FILTER (?Target_Organ in ( 'heart', 'ovary', 'brain', 'urethra', 'esophagus', 'skin of body', 'lung', 'liver', 
                                'lower urinary tract', 'urinary tract', 'muscle organ','gallbladder', 'colon', 'kidney', 
                                'large intestine' ,'small intestine', 'stomach', 'spleen', 'urinary bladder', 
                                'penis', 'clitoris', 'pancreas'))       
}
ORDER BY ?Neuron_ID ?A_Label ?B_Label
LIMIT 9999

Neuron_ID,A_IRI,A_Label,B_IRI,B_Label,Target_Organ
http://uri.interlex.org/tgbugs/uris/readable/neuron-type-aacar-11,http://purl.obolibrary.org/obo/UBERON_0005363,inferior vagus X ganglion,http://purl.obolibrary.org/obo/UBERON_0002348,epicardium,heart
http://uri.interlex.org/tgbugs/uris/readable/neuron-type-aacar-11,http://purl.obolibrary.org/obo/UBERON_0005363,inferior vagus X ganglion,http://purl.obolibrary.org/obo/UBERON_0002084,heart left ventricle,heart
http://uri.interlex.org/tgbugs/uris/readable/neuron-type-aacar-11,http://purl.obolibrary.org/obo/UBERON_0005363,inferior vagus X ganglion,http://purl.obolibrary.org/obo/UBERON_0002079,left cardiac atrium,heart
http://uri.interlex.org/tgbugs/uris/readable/neuron-type-aacar-11,http://purl.obolibrary.org/obo/UBERON_0005363,inferior vagus X ganglion,http://purl.obolibrary.org/obo/UBERON_0002078,right cardiac atrium,heart
http://uri.interlex.org/tgbugs/uris/readable/neuron-type-aacar-13,http://purl.obolibrary.org/obo/UBERON_0002844,eighth cervical dorsal root ganglion,http://purl.obolibrary.org/obo/UBERON_0002348,epicardium,heart
http://uri.interlex.org/tgbugs/uris/readable/neuron-type-aacar-13,http://purl.obolibrary.org/obo/UBERON_0002844,eighth cervical dorsal root ganglion,http://purl.obolibrary.org/obo/UBERON_0002084,heart left ventricle,heart
http://uri.interlex.org/tgbugs/uris/readable/neuron-type-aacar-13,http://purl.obolibrary.org/obo/UBERON_0002844,eighth cervical dorsal root ganglion,http://purl.obolibrary.org/obo/UBERON_0002080,heart right ventricle,heart
http://uri.interlex.org/tgbugs/uris/readable/neuron-type-aacar-13,http://purl.obolibrary.org/obo/UBERON_0002848,fifth thoracic dorsal root ganglion,http://purl.obolibrary.org/obo/UBERON_0002348,epicardium,heart
http://uri.interlex.org/tgbugs/uris/readable/neuron-type-aacar-13,http://purl.obolibrary.org/obo/UBERON_0002848,fifth thoracic dorsal root ganglion,http://purl.obolibrary.org/obo/UBERON_0002084,heart left ventricle,heart
http://uri.interlex.org/tgbugs/uris/readable/neuron-type-aacar-13,http://purl.obolibrary.org/obo/UBERON_0002848,fifth thoracic dorsal root ganglion,http://purl.obolibrary.org/obo/UBERON_0002080,heart right ventricle,heart


## Example 9: Neurons with Axonal Paths
This is a query where we want to produce an adjacency matrix to retrieve the axonal path of the neuron populations. Each axonal path forms a partial order of locations a neuron population forms between its origin(s) and destination(s). 
* <b>NOTE:</b> The result of this query can be used to visualize the axonal paths using different visualization tools. For example, you can use the result of this query to generate digraphs in dot format and visualize the axonal paths for the neuron populations in GraphViz. 


In [12]:
# Query to generate the adjacency matrix representing the axonal paths of the neuron populations.

SELECT DISTINCT ?Neuron_IRI ?Neuron_Label ?V1 ?V1_Label ?V2 ?V2_Label
WHERE 
{

    # Note: This query currently works for the NLP curated neuron populations.
    # We will update the query for other populations such as the populations from ApINATOMY models. 
    ?Neuron_IRI rdfs:subClassOf ilxtr:NeuronSparcNlp.

    ?Neuron_IRI ilxtr:neuronPartialOrder ?PO .
        ?PO (rdf:rest|rdf:first)* ?r1 .
        ?r1 (rdf:rest|rdf:first)* ?r2 .
        ?r1 rdf:first ?V1 .
        ?r2 rdf:first ?V2 .
        ?V1 rdf:type owl:Class .
        ?V2 rdf:type owl:Class .

        FILTER (?mediator = ?r1)  # draw only from the same partial order
        ?mediator rdf:first ?V1 .  # car
        ?mediator rdf:rest+/rdf:first/rdf:first ?V2 .  # caadr

    ?V1 rdfs:label ?V1_Label. ?V2 rdfs:label ?V2_Label.
    ?Neuron_IRI rdfs:label ?Neuron_Label.

    FILTER (?V1 != ?V2).

   # FILTER (CONTAINS(STR(?Neuron_IRI), 'semves/2')).

}
ORDER BY ?Neuron_IRI ?V1 ?V2 
LIMIT 10000

Neuron_IRI,Neuron_Label,V1,V1_Label,V2,V2_Label
http://uri.interlex.org/tgbugs/uris/readable/sparc-nlp/mmset1/1,Nucleus of the pudendal nerve to urethral sphincter via pudendal nerve,http://purl.obolibrary.org/obo/UBERON_0011390,pudendal nerve,http://purl.obolibrary.org/obo/UBERON_0004917,urethral sphincter
http://uri.interlex.org/tgbugs/uris/readable/sparc-nlp/mmset1/1,Nucleus of the pudendal nerve to urethral sphincter via pudendal nerve,http://purl.obolibrary.org/obo/UBERON_0022278,nucleus of pudendal nerve,http://purl.obolibrary.org/obo/UBERON_0011390,pudendal nerve
http://uri.interlex.org/tgbugs/uris/readable/sparc-nlp/mmset1/10,Superior salivatory nucleus to pterygopalatine ganglion via facial nerve via greater petrosal nerve via vidian nerve,http://purl.obolibrary.org/obo/UBERON_0001647,facial nerve,http://uri.interlex.org/base/ilx_0793702,Greater petrosal nerve
http://uri.interlex.org/tgbugs/uris/readable/sparc-nlp/mmset1/10,Superior salivatory nucleus to pterygopalatine ganglion via facial nerve via greater petrosal nerve via vidian nerve,http://purl.obolibrary.org/obo/UBERON_0002149,superior salivatory nucleus,http://purl.obolibrary.org/obo/UBERON_0001647,facial nerve
http://uri.interlex.org/tgbugs/uris/readable/sparc-nlp/mmset1/10,Superior salivatory nucleus to pterygopalatine ganglion via facial nerve via greater petrosal nerve via vidian nerve,http://purl.obolibrary.org/obo/UBERON_0018412,vidian nerve,http://purl.obolibrary.org/obo/UBERON_0003962,pterygopalatine ganglion
http://uri.interlex.org/tgbugs/uris/readable/sparc-nlp/mmset1/10,Superior salivatory nucleus to pterygopalatine ganglion via facial nerve via greater petrosal nerve via vidian nerve,http://uri.interlex.org/base/ilx_0793702,Greater petrosal nerve,http://purl.obolibrary.org/obo/UBERON_0018412,vidian nerve
http://uri.interlex.org/tgbugs/uris/readable/sparc-nlp/mmset1/11,T2-T6 intermediolateral cell column of spinal cord to cervicothoracic ganglion-T5 ganglion via T2-T6 white communicating ramus via T2-T6 sympathetic chain,http://uri.interlex.org/base/ilx_0793208,white communicating ramus of second thoracic spinal nerve,http://uri.interlex.org/base/ilx_0793335,T5 sympathetic chain
http://uri.interlex.org/tgbugs/uris/readable/sparc-nlp/mmset1/11,T2-T6 intermediolateral cell column of spinal cord to cervicothoracic ganglion-T5 ganglion via T2-T6 white communicating ramus via T2-T6 sympathetic chain,http://uri.interlex.org/base/ilx_0793208,white communicating ramus of second thoracic spinal nerve,http://uri.interlex.org/base/ilx_0793336,T6 sympathetic chain
http://uri.interlex.org/tgbugs/uris/readable/sparc-nlp/mmset1/11,T2-T6 intermediolateral cell column of spinal cord to cervicothoracic ganglion-T5 ganglion via T2-T6 white communicating ramus via T2-T6 sympathetic chain,http://uri.interlex.org/base/ilx_0793208,white communicating ramus of second thoracic spinal nerve,http://uri.interlex.org/base/ilx_0793617,T2 sympathetic chain
http://uri.interlex.org/tgbugs/uris/readable/sparc-nlp/mmset1/11,T2-T6 intermediolateral cell column of spinal cord to cervicothoracic ganglion-T5 ganglion via T2-T6 white communicating ramus via T2-T6 sympathetic chain,http://uri.interlex.org/base/ilx_0793208,white communicating ramus of second thoracic spinal nerve,http://uri.interlex.org/base/ilx_0793618,T3 sympathetic chain


## Example 10: Neurons with Forward Connections
We often need to know whether a neuron population is connected to other neuron populations through its synaptic terminal. SCKAN uses ilxtr:hasForwardConnection predicate to connect the two populations. Whenever we have such connection, that basically means that a preganglionic neuron population has a terminal location (i.e., axon terminal or axon sensory terminal) that is the origin of a postganglionic neuron population.

In [13]:
# Query to list the neuron populations with their forward connections with other populations

SELECT DISTINCT ?Neuron_1_IRI ?Neuron_1_Label 
                ?Neuron_2_IRI ?Neuron_2_Label
WHERE 
{
    # We are only interested in connection forming neuron types i.e, neurons that form some connection 
    # between their origin and destination regions
   ?Neuron_1_IRI ilxtr:hasSomaLocation ?Neuron_1_A;
               (ilxtr:hasAxonTerminalLocation | ilxtr:hasAxonSensoryLocation) ?Nuron_1_B .
    
   ?Neuron_2_IRI ilxtr:hasSomaLocation ?Neuron_2_A;
               (ilxtr:hasAxonTerminalLocation | ilxtr:hasAxonSensoryLocation) ??Neuron_2_B .
    
   ## To restrict the results within the NLP curated neuron populations.
   #  ?Neuron_1_IRI rdfs:subClassOf ilxtr:NeuronSparcNlp .
   #  ?Neuron_2_IRI rdfs:subClassOf ilxtr:NeuronSparcNlp .

   
   ?Neuron_1_IRI ilxtr:hasForwardConnection ?Neuron_2_IRI .
   
   ?Neuron_1_IRI rdfs:label ?Neuron_1_Label .
   ?Neuron_2_IRI rdfs:label ?Neuron_2_Label . 

  ## Filter results based on a particular neuron population
  # FILTER(CONTAINS(STR(?Neuron_1_IRI), 'prostate')).  
  # FILTER(CONTAINS(STR(?Neuron_2_IRI), 'mmset4'))
   
} 
ORDER BY ?Neuron_1_IRI ?Neuron_2_IRI

Neuron_1_IRI,Neuron_1_Label,Neuron_2_IRI,Neuron_2_Label
http://uri.interlex.org/tgbugs/uris/readable/neuron-type-aacar-10a,neuron type aacar 10a,http://uri.interlex.org/tgbugs/uris/readable/neuron-type-aacar-7a,neuron type aacar 7a
http://uri.interlex.org/tgbugs/uris/readable/neuron-type-aacar-10a,neuron type aacar 10a,http://uri.interlex.org/tgbugs/uris/readable/neuron-type-aacar-8a,neuron type aacar 8a
http://uri.interlex.org/tgbugs/uris/readable/neuron-type-aacar-10v,neuron type aacar 10v,http://uri.interlex.org/tgbugs/uris/readable/neuron-type-aacar-7v,neuron type aacar 7v
http://uri.interlex.org/tgbugs/uris/readable/neuron-type-aacar-10v,neuron type aacar 10v,http://uri.interlex.org/tgbugs/uris/readable/neuron-type-aacar-8v,neuron type aacar 8v
http://uri.interlex.org/tgbugs/uris/readable/neuron-type-aacar-11,neuron type aacar 11,http://uri.interlex.org/tgbugs/uris/readable/neuron-type-aacar-1,neuron type aacar 1
http://uri.interlex.org/tgbugs/uris/readable/neuron-type-aacar-12,neuron type aacar 12,http://uri.interlex.org/tgbugs/uris/readable/neuron-type-aacar-2i,neuron type aacar 2i
http://uri.interlex.org/tgbugs/uris/readable/neuron-type-aacar-12,neuron type aacar 12,http://uri.interlex.org/tgbugs/uris/readable/neuron-type-aacar-2m,neuron type aacar 2m
http://uri.interlex.org/tgbugs/uris/readable/neuron-type-aacar-2i,neuron type aacar 2i,http://uri.interlex.org/tgbugs/uris/readable/neuron-type-aacar-6,neuron type aacar 6
http://uri.interlex.org/tgbugs/uris/readable/neuron-type-aacar-2m,neuron type aacar 2m,http://uri.interlex.org/tgbugs/uris/readable/neuron-type-aacar-6,neuron type aacar 6
http://uri.interlex.org/tgbugs/uris/readable/neuron-type-aacar-4,neuron type aacar 4,http://uri.interlex.org/tgbugs/uris/readable/neuron-type-aacar-10a,neuron type aacar 10a
